In [2]:
import pandas as pd
import numpy as np

import gc
from tqdm import tqdm
import time
import os
import warnings
from datetime import datetime

from chicken_dinner.pubgapi import PUBG
from chicken_dinner.constants import map_dimensions

import pymysql
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

warnings.filterwarnings(action = 'ignore')

path = os.getcwd()
data_path = os.path.join(path, 'data')

## Functions for web scrap supply object

In [7]:
def get_telemetry(match_id, map_name):

    current_match = pubg.match(match_id)
    telemetry = current_match.get_telemetry()

    if map_name == 'Tiger_Main':
        mapx, mapy = map_dimensions['Desert_Main']
    else:
        mapx, mapy = map_dimensions[map_name]
        
    return telemetry, mapy

In [8]:
def change_date_format(timestamp):
    timestamp = pd.Timestamp(timestamp).to_pydatetime().replace(tzinfo=None)
    return timestamp

In [9]:
def rename_column_names(column_names):
    if len(column_names.split('.')) > 2:
        return ('_').join(column_names.split('.')[-2:])
    else:
        return column_names.split('.')[-1]

In [10]:
def get_weapon_name(items):
    try:
        return [item['item_id'] for item in items if item['sub_category'] == 'Main'][0]
    except IndexError:
        return np.NaN

In [11]:
def get_supply_df(telemetry, mapy):

    care_package_land_sample_df = pd.DataFrame()
    
    care_package_lands = telemetry.filter_by('log_care_package_land')

    # get care paackage dataframe of a match
    for care_package_land in care_package_lands:
        care_package_land_sample_df = pd.concat([care_package_land_sample_df, pd.json_normalize(care_package_land.to_dict())], axis = 0, ignore_index = True)

    # match table을 참조하는 Foreign key column
    care_package_land_sample_df['match_id'] = match_id

    # rename columns
    care_package_land_sample_df.columns = list(map(lambda x: rename_column_names(x), care_package_land_sample_df.columns.tolist()))
    care_package_land_sample_df = care_package_land_sample_df.rename(columns = {'_D': 'log_created_time'})

    care_package_land_sample_df.loc[:, 'items'] = care_package_land_sample_df.loc[:, 'items'].apply(lambda x: get_weapon_name(x))
    care_package_land_sample_df.loc[care_package_land_sample_df['items'] == 'None', 'items'] = np.nan
    
    # map 시각화를 위해 y axis dimension 변경
    care_package_land_sample_df.loc[:, 'location_y'] = mapy - care_package_land_sample_df.loc[:, 'location_y']

    # date format 변경
    care_package_land_sample_df.loc[:, 'log_created_time'] = care_package_land_sample_df.loc[:, 'log_created_time'].apply(lambda x: change_date_format(x))
    
    # 불필요한 컬럼 제거
    care_package_land_sample_df = care_package_land_sample_df.drop('_T', axis = 1)

    return care_package_land_sample_df

## Get supply table by match id's from match_data

In [12]:
%%time

# PUBG api authorize
pubg = PUBG(api_key, shard = 'kakao')

supply_df = pd.DataFrame()
match_json = pd.read_json(os.path.join(data_path, 'match_data.json'))

for idx in tqdm(range(len(match_json))):

    match_id = match_json['data.id'][idx]
    map_name = match_json['data.attributes.mapName'][idx]
    
    telemetry, mapy = get_telemetry(match_id, map_name)
    
    supply_df = pd.concat([supply_df, get_supply_df(telemetry, mapy)], axis = 0, ignore_index = True)
    
print(f'shape of Supply df: {supply_df.shape}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [03:18<00:00,  4.23s/it]


## Insert into database

In [ ]:
# # local
# user = 'root'
# password = 'mysql'
# host = 'localhost'
# port = 3306
# database = 'pubg'

# gcp
user = ''
password = ''
host = ''
port = 
database = ''

In [ ]:
def insert_data_to_db(data, table_name, connection, if_exist = 'append'):
    
    data.to_sql(index = False,
                name = table_name,
                con = connection,
                if_exists = if_exist,
                method = 'multi')

In [ ]:
%%time

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}', encoding = 'utf-8')
engine_conn = engine_connect()

insert_data_to_db(damage_df, 'damage', engine_conn)
engine_conn.close()